### Face Recognition using SVM

In [ ]:
# standard imports
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import scipy

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)

In [ ]:
print(faces.target_names)
print(faces.images.shape)

In [ ]:
fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

We could proceed by simply
using each pixel value as a feature, but often it is more effective \
to use some sort of
preprocessor to extract more meaningful features

In [ ]:
# Using PCA to extract features for this case
from sklearn.svm import SVC
from sklearn.decomposition import PCA  # Our preprocessor
from sklearn.pipeline import make_pipeline

pca = PCA(n_components=150, whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    faces.data, faces.target, random_state=42)

In [ ]:
param_grid = {'svc__C': [1, 5, 10, 50],
              'svc__gamma': [0.0001, 0.0005, 0.001, 0.005]}

grid = GridSearchCV(model, param_grid)
%time grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
model = grid.best_estimator_
yfit = model.predict(X_test)

In [ ]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(X_test[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == y_test[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14)

Out of this small sample, our optimal estimator mislabeled only two faces.\
We can get a better sense of our esti‐
mator’s performance using the classification report, which lists recovery statistics label by label:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, yfit, target_names=faces.target_names))

We can also display the confusion matrix between these classes

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_test, yfit)

# visualizing confusion matrix using a heatmap
sns.heatmap(mat.T, square = True, annot=True, fmt='d', cbar = False,  xticklabels=faces.target_names,
yticklabels=faces.target_names)

plt.xlabel('True Label')
plt.xlabel('Prediccted Label');